Every possible analysis on VOLD crossovers for the 5 min chart timeframe

1. VOLD crossover from below 0 to above 0
2. VOLD crossover from below 0 to above 0 before 9:30 AM
3. VOLD crossover from below 0 to above 0 and carry over the trade to max profit or until VOLD crosses down below 0 again.

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Function to filter buy signals based on VOLD crossover
def filter_signals_vold_crossover(data, time_limit=None):
    if 'Normalized VOLD Ratio' not in data.columns:
        print("Warning: 'Normalized VOLD Ratio' column not found in the data. Skipping this file.")
        return None

    buy_signals = data[(data['Normalized VOLD Ratio'].shift(1) < 0) & (data['Normalized VOLD Ratio'] > 0)]
    
    if time_limit:
        buy_signals['time_ist'] = pd.to_datetime(buy_signals['time'], unit='s') + timedelta(hours=5, minutes=30)
        buy_signals = buy_signals[buy_signals['time_ist'].dt.time <= time_limit]

    return buy_signals

# Function to calculate charges
def calculate_charges(entry_price, exit_price, quantity):
    brokerage = 40  # total brokerage for one complete buy and sell
    stt_ctt = 0.00125 * exit_price * quantity
    transaction_charges = 0.000495 * (entry_price + exit_price) * quantity
    gst = 0.18 * (brokerage + transaction_charges)
    sebi_charges = 10 / 10**7 * (entry_price + exit_price) * quantity
    stamp_charges = 0.00003 * entry_price * quantity
    total_charges = brokerage + stt_ctt + transaction_charges + gst + sebi_charges + stamp_charges
    return total_charges

# Function to simulate day trades with error margin
def simulate_day_trades(buy_signals, minute_data, symbol, profit_target=0.02, stop_loss=0.01, min_stop_loss=0.005, error_margin=0.01, carry_over=False):
    results = []

    for index, buy_signal in buy_signals.iterrows():
        entry_price = buy_signal['close'] * (1 + error_margin)  # Adjust entry price with error margin
        entry_time = buy_signal['time'] + 300  # Add 5 minutes to the buy signal time to get the entry time
        
        # Convert entry_time from epoch to datetime
        entry_datetime = pd.to_datetime(entry_time, unit='s')
        entry_datetime_ist = entry_datetime + timedelta(hours=5, minutes=30)  # Convert to IST
        
        profit_price = entry_price * (1 + float(profit_target))
        stop_price = entry_price * (1 - max(float(stop_loss), min_stop_loss))  # Ensure stop loss is at least min_stop_loss
        
        trade_result = {
            'signal_time': buy_signal['time'],
            'entry_time': entry_time,
            'entry_price': entry_price,
            'Target Profit': profit_target,
            'Stop Loss': stop_loss,
            'Profit Price': profit_price,
            'Stop Price': stop_price,
            'exit_time': None,
            'exit_price': None,
            'charges': None,
            'profit_before_charges': None,
            'profit': None,
            'exit_reason': None,
            'SOSI': buy_signal.get('SOSI'),
            'Normalized VOLD Ratio': buy_signal.get('Normalized VOLD Ratio'),
            'Normalized Theta': buy_signal.get('Normalized Theta')
        }
        
        # Filter subsequent data for the same day only
        trade_date = pd.to_datetime(entry_time, unit='s').date()
        subsequent_data = minute_data[(minute_data['time'] > entry_time) & 
                                      (pd.to_datetime(minute_data['time'], unit='s').dt.date == trade_date)]
        
        for _, row in subsequent_data.iterrows():
            if row['high'] >= profit_price:
                trade_result['exit_time'] = row['time']
                trade_result['exit_price'] = profit_price
                trade_result['exit_reason'] = 'Target Hit'
                break
            if row['low'] <= stop_price:
                trade_result['exit_time'] = row['time']
                trade_result['exit_price'] = stop_price
                trade_result['exit_reason'] = 'Stop Loss Hit'
                break
            if carry_over and 'Normalized VOLD Ratio' in row and row['Normalized VOLD Ratio'] < 0:
                trade_result['exit_time'] = row['time']
                trade_result['exit_price'] = row['close']
                trade_result['exit_reason'] = 'VOLD Cross Below 0'
                break
        else:
            # If no target or stop loss hit, exit at the last price of the day
            trade_result['exit_reason'] = 'End of Day Exit'
            if not subsequent_data.empty:
                last_row = subsequent_data.iloc[-1]
                trade_result['exit_time'] = last_row['time']
                trade_result['exit_price'] = last_row['close']
        
        if trade_result['exit_price'] is not None:
            quantity = 25 if 'NIFTY' in symbol else 15 if 'BANKNIFTY' in symbol else 40 if 'FINNIFTY' in symbol else 10
            gross_profit = (trade_result['exit_price'] - trade_result['entry_price']) * quantity
            charges = calculate_charges(trade_result['entry_price'], trade_result['exit_price'], quantity)
            net_profit = gross_profit - charges
            trade_result['charges'] = charges
            trade_result['profit_before_charges'] = gross_profit
            trade_result['profit'] = net_profit

        results.append(trade_result)
    
    results_df = pd.DataFrame(results)
    
    if results_df.empty:
        print("No trades were executed.")
        return pd.DataFrame(columns=['signal_time', 'entry_time', 'entry_price', 'Target Profit', 'Stop Loss', 
                                     'Profit Price', 'Stop Price', 'exit_time', 'exit_price', 'charges', 
                                     'profit_before_charges', 'profit', 'exit_reason', 'SOSI', 
                                     'Normalized VOLD Ratio', 'Normalized Theta'])
    
    return results_df.dropna(subset=['entry_price'])

# Function to analyze profit and stop loss percentages
def analyze_profit_and_stop_loss(buy_signals, minute_data, folder_name, profit_percentages, stop_loss_percentages, min_stop_loss=0.005, error_margin=0.01, carry_over=False):
    analysis_results = []
    finalresults = []
    for profit_target in profit_percentages:
        for stop_loss in stop_loss_percentages:
            trade_results = simulate_day_trades(buy_signals, minute_data, folder_name, profit_target, stop_loss, min_stop_loss, error_margin, carry_over=carry_over)
            if not trade_results.empty:
                total_profit = trade_results['profit'].sum()
                charges = trade_results['charges'].sum()
                trade_time=trade_results['signal_time']
                profit_before_charges = trade_results['profit_before_charges'].sum()
                analysis_results.append((trade_time,profit_target, stop_loss, charges, profit_before_charges, total_profit))
                finalresults.append(trade_results)
    # Combine all trade results into a single DataFrame
    if finalresults:
        combined_trade_results = pd.concat(finalresults, ignore_index=True)
    else:
        combined_trade_results = pd.DataFrame(columns=['signal_time', 'entry_time', 'entry_price', 'Target Profit', 
                                                       'Stop Loss', 'Profit Price', 'Stop Price', 'exit_time', 
                                                       'exit_price', 'charges', 'profit_before_charges', 'profit', 
                                                       'exit_reason', 'SOSI', 'Normalized VOLD Ratio', 'Normalized Theta'])
 
    return pd.DataFrame(analysis_results, columns=['Signal Time','Profit Target', 'Stop Loss', 'Charges', 'Profit Before Charges', 'Total Profit']), combined_trade_results

# Function to summarize trade results
def summarize_trade_results(trade_results):
    total_trades = len(trade_results)
    target_hit = len(trade_results[trade_results['exit_reason'] == 'Target Hit'])
    stop_loss_hit = len(trade_results[trade_results['exit_reason'] == 'Stop Loss Hit'])
    end_of_day_exit = len(trade_results[trade_results['exit_reason'] == 'End of Day Exit'])
    max_drawdown = trade_results['profit'].min() if not trade_results.empty else 0
    summary = {
        'Total Trades': total_trades,
        'Target Hit': target_hit,
        'Stop Loss Hit': stop_loss_hit,
        'End of Day Exit': end_of_day_exit,
        'Max Drawdown': max_drawdown
    }
    return summary

# Define profit percentages and stop loss percentages to analyze
profit_percentages = [i / 100 for i in range(1, 200)]
stop_loss_percentages = [i / 100 for i in range(1, 20)]
profit_percentages_carryover = [i / 100 for i in range(1, 1000)]
stop_loss_percentages_carryover = [i / 100 for i in range(1, 20)]

# Directories containing the data
directories = ['June 2024/NIFTY', 'June 2024/BANKNIFTY', 'June 2024/FINNIFTY']  # Change the directories as needed

# Function to check if the date is within the week of expiration
def is_within_expiry_week(expiry_date, timestamp, folder_name, days_before_expiry=5):
    # Assuming the date is the 7th to 12th characters in the string
    if folder_name == 'NIFTY':
        date_part = expiry_date[9:15]
    if folder_name == 'BANKNIFTY':
        date_part = expiry_date[13:19]
    if folder_name == 'FINNIFTY':
        date_part = expiry_date[12:18]
    if folder_name == 'SENSEX':
        date_part = expiry_date[7:13]
    
    expiry_datetime = pd.to_datetime(date_part, format='%y%m%d')
    signal_datetime = pd.to_datetime(timestamp, unit='s')
    return (expiry_datetime - signal_datetime).days <= days_before_expiry

# Process each directory
for directory in directories:
    pnl_results_below_9_30 = []
    pnl_results_eod = []
    pnl_results_carry_over = []
    trade_results_combined_below_9_30 = []
    trade_results_combined_eod = []
    trade_results_combined_carry_over = []
    all_combined_results_below_9_30 = []
    all_combined_results_eod = []
    all_combined_results_carry_over = []
    folder_name = os.path.basename(directory)
    for subdir, _, files in os.walk(directory):
        if subdir == directory:
            continue  # Skip the top-level directory itself
        for filename in files:
            if ', 5.csv' in filename:
                minute_5_path = os.path.join(subdir, filename)
                minute_1_path = minute_5_path.replace(', 5.csv', ', 1.csv')
                
                if os.path.exists(minute_1_path):
                    print(f"Processing {filename} in {folder_name}...")
                    
                    # Extract expiry date from filename
                    expiry_date = filename.split(' ')[0].split(',')[0]
                    
                    # Load the CSV files
                    minute_data_5 = pd.read_csv(minute_5_path)
                    minute_data_1 = pd.read_csv(minute_1_path)
                    
                    # Scenario 1: Filter buy signals before 9:30 AM IST
                    buy_signals_below_9_30 = filter_signals_vold_crossover(minute_data_5, time_limit=datetime.strptime('09:30:00', '%H:%M:%S').time())
                    if buy_signals_below_9_30 is None or buy_signals_below_9_30.empty:
                        print(f"No buy signals found before 9:30 AM IST for {filename}. Skipping...")
                        continue
                    
                    # Scenario 2: Filter buy signals with no time limit
                    buy_signals_no_time_limit = filter_signals_vold_crossover(minute_data_5)
                    if buy_signals_no_time_limit is None or buy_signals_no_time_limit.empty:
                        print(f"No buy signals found for {filename}. Skipping...")
                        continue
                    
                    # Perform the analysis for each scenario
                    # Scenario 1: Below 9:30 AM IST
                    analysis_results_below_9_30, trade_results_below_9_30 = analyze_profit_and_stop_loss(
                        buy_signals_below_9_30, minute_data_1, folder_name, profit_percentages, stop_loss_percentages
                    )
                    combined_results_below_9_30 = pd.merge(buy_signals_below_9_30, trade_results_below_9_30, left_on='time', right_on='signal_time', how='inner')
                    combined_results_below_9_30 = pd.merge(combined_results_below_9_30, minute_data_5, left_on='time', right_on='time', suffixes=('_buy', '_original'))
                    all_combined_results_below_9_30.append(combined_results_below_9_30)
                    pnl_results_below_9_30.append(analysis_results_below_9_30)
                    trade_results_combined_below_9_30.append(trade_results_below_9_30)
                    
                    # Scenario 2: End of day exits
                    analysis_results_eod, trade_results_eod = analyze_profit_and_stop_loss(
                        buy_signals_no_time_limit, minute_data_1, folder_name, profit_percentages, stop_loss_percentages
                    )
                    combined_results_eod = pd.merge(buy_signals_no_time_limit, trade_results_eod, left_on='time', right_on='signal_time', how='inner')
                    combined_results_eod = pd.merge(combined_results_eod, minute_data_5, left_on='time', right_on='time', suffixes=('_buy', '_original'))
                    all_combined_results_eod.append(combined_results_eod)
                    pnl_results_eod.append(analysis_results_eod)
                    trade_results_combined_eod.append(trade_results_eod)
                    
                    # Scenario 3: Carry over trades
                    analysis_results_carry_over, trade_results_carry_over = analyze_profit_and_stop_loss(
                        buy_signals_no_time_limit, minute_data_1, folder_name, profit_percentages_carryover, stop_loss_percentages_carryover, carry_over=True
                    )
                    combined_results_carry_over = pd.merge(buy_signals_no_time_limit, trade_results_carry_over, left_on='time', right_on='signal_time', how='inner')
                    combined_results_carry_over = pd.merge(combined_results_carry_over, minute_data_5, left_on='time', right_on='time', suffixes=('_buy', '_original'))
                    all_combined_results_carry_over.append(combined_results_carry_over)
                    pnl_results_carry_over.append(analysis_results_carry_over)
                    trade_results_combined_carry_over.append(trade_results_carry_over)

    # Combine and save results for signals before 9:30 AM IST
    if pnl_results_below_9_30:
        combined_pnl_results_below_9_30 = pd.concat(pnl_results_below_9_30, ignore_index=True)
        combined_pnl_results_below_9_30 = combined_pnl_results_below_9_30.groupby(['Profit Target', 'Stop Loss'], as_index=False)['Total Profit'].sum()
        combined_trade_results_below_9_30 = pd.concat(trade_results_combined_below_9_30, ignore_index=True)
        combined_all_results_below_9_30 = pd.concat(all_combined_results_below_9_30, ignore_index=True)

        output_dir_below_9_30 = f'Outputs/DayTrade/SOSI/5min/{folder_name}_below_9_30'
        os.makedirs(output_dir_below_9_30, exist_ok=True)
        
        combined_pnl_results_below_9_30.to_csv(f'{output_dir_below_9_30}/{folder_name}_pnl_analysis.csv', index=False)
        combined_trade_results_below_9_30.to_csv(f'{output_dir_below_9_30}/{folder_name}_trade_results.csv', index=False)
        combined_all_results_below_9_30.to_csv(f'{output_dir_below_9_30}/{folder_name}_all_combined_results.csv', index=False)
        
        best_combination_below_9_30 = combined_pnl_results_below_9_30.loc[combined_pnl_results_below_9_30['Total Profit'].idxmax()]
        best_combination_df_below_9_30 = pd.DataFrame([best_combination_below_9_30])
        best_combination_df_below_9_30.to_csv(f'{output_dir_below_9_30}/{folder_name}_best_combination.csv', index=False)
        
        top_40_combinations_below_9_30 = combined_pnl_results_below_9_30.sort_values(by='Total Profit', ascending=False).head(40)
        top_40_combinations_below_9_30.to_csv(f'{output_dir_below_9_30}/{folder_name}_top_40_combinations.csv', index=False)
        
        best_trade_results_below_9_30 = combined_trade_results_below_9_30[(combined_trade_results_below_9_30['Target Profit'] == best_combination_below_9_30['Profit Target']) & 
                                                                         (combined_trade_results_below_9_30['Stop Loss'] == best_combination_below_9_30['Stop Loss'])]
        trade_summary_below_9_30 = summarize_trade_results(best_trade_results_below_9_30)
        trade_summary_df_below_9_30 = pd.DataFrame([trade_summary_below_9_30])
        trade_summary_df_below_9_30.to_csv(f'{output_dir_below_9_30}/{folder_name}_trade_summary.csv', index=False)
        
        labels = ['Target Hit', 'Stop Loss Hit', 'End of Day Exit']
        sizes = [trade_summary_below_9_30['Target Hit'], trade_summary_below_9_30['Stop Loss Hit'], trade_summary_below_9_30['End of Day Exit']]
        colors = ['lightgreen', 'lightcoral', 'lightskyblue']
        plt.figure(figsize=(8, 8))
        plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
        plt.title(f'Trade Summary for Best PNL Combination (Before 9:30 AM)\n{folder_name}')
        plt.axis('equal')
        plt.show()
        plt.savefig(f'{output_dir_below_9_30}/{folder_name}_trade_summary_pie_chart.png')
        plt.close()
        
        plt.figure(figsize=(12, 8))
        plt.barh(['Total Trades'], [trade_summary_below_9_30['Total Trades']], color='skyblue')
        plt.xlabel('Count')
        plt.ylabel('Total Trades')
        plt.title(f'Total Trades for Best PNL Combination (Before 9:30 AM)\n{folder_name}')
        plt.tight_layout()
        plt.show()
        plt.savefig(f'{output_dir_below_9_30}/{folder_name}_total_trades_bar_chart.png')
        plt.close()
    
        plt.figure(figsize=(12, 8))
        plt.barh(['Max Drawdown'], [trade_summary_below_9_30['Max Drawdown']], color='red')
        plt.xlabel('Max Drawdown')
        plt.ylabel('Max Drawdown Value')
        plt.title(f'Max Drawdown for Best PNL Combination (Before 9:30 AM)\n{folder_name}')
        plt.tight_layout()
        plt.show()
        plt.savefig(f'{output_dir_below_9_30}/{folder_name}_max_drawdown_bar_chart.png')
        plt.close()
    
        plt.figure(figsize=(12, 8))
        plt.barh(top_40_combinations_below_9_30['Profit Target'].astype(str) + " / " + top_40_combinations_below_9_30['Stop Loss'].astype(str),
                 top_40_combinations_below_9_30['Total Profit'], color='skyblue')
        plt.xlabel('Total Profit')
        plt.ylabel('Profit Target / Stop Loss')
        plt.title(f'Top 40 Profit Target and Stop Loss Combinations (Before 9:30 AM)\n{folder_name}')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()
        plt.savefig(f'{output_dir_below_9_30}/{folder_name}_top_40_combinations.png')
        plt.close()

    # Combine and save results for end of day exits
    if pnl_results_eod:
        combined_pnl_results_eod = pd.concat(pnl_results_eod, ignore_index=True)
        combined_pnl_results_eod = combined_pnl_results_eod.groupby(['Profit Target', 'Stop Loss'], as_index=False)['Total Profit'].sum()
        combined_trade_results_eod = pd.concat(trade_results_combined_eod, ignore_index=True)
        combined_all_results_eod = pd.concat(all_combined_results_eod, ignore_index=True)

        output_dir_eod = f'Outputs/DayTrade/SOSI/5min/{folder_name}_eod'
        os.makedirs(output_dir_eod, exist_ok=True)
        
        combined_pnl_results_eod.to_csv(f'{output_dir_eod}/{folder_name}_pnl_analysis.csv', index=False)
        combined_trade_results_eod.to_csv(f'{output_dir_eod}/{folder_name}_trade_results.csv', index=False)
        combined_all_results_eod.to_csv(f'{output_dir_eod}/{folder_name}_all_combined_results.csv', index=False)
        
        best_combination_eod = combined_pnl_results_eod.loc[combined_pnl_results_eod['Total Profit'].idxmax()]
        best_combination_df_eod = pd.DataFrame([best_combination_eod])
        best_combination_df_eod.to_csv(f'{output_dir_eod}/{folder_name}_best_combination.csv', index=False)
        
        top_40_combinations_eod = combined_pnl_results_eod.sort_values(by='Total Profit', ascending=False).head(40)
        top_40_combinations_eod.to_csv(f'{output_dir_eod}/{folder_name}_top_40_combinations.csv', index=False)
        
        best_trade_results_eod = combined_trade_results_eod[(combined_trade_results_eod['Target Profit'] == best_combination_eod['Profit Target']) & 
                                                            (combined_trade_results_eod['Stop Loss'] == best_combination_eod['Stop Loss'])]
        trade_summary_eod = summarize_trade_results(best_trade_results_eod)
        trade_summary_df_eod = pd.DataFrame([trade_summary_eod])
        trade_summary_df_eod.to_csv(f'{output_dir_eod}/{folder_name}_trade_summary.csv', index=False)
        
        labels = ['Target Hit', 'Stop Loss Hit', 'End of Day Exit']
        sizes = [trade_summary_eod['Target Hit'], trade_summary_eod['Stop Loss Hit'], trade_summary_eod['End of Day Exit']]
        colors = ['lightgreen', 'lightcoral', 'lightskyblue']
        plt.figure(figsize=(8, 8))
        plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
        plt.title(f'Trade Summary for Best PNL Combination (EOD)\n{folder_name}')
        plt.axis('equal')
        plt.show()
        plt.savefig(f'{output_dir_eod}/{folder_name}_trade_summary_pie_chart.png')
        plt.close()
        
        plt.figure(figsize=(12, 8))
        plt.barh(['Total Trades'], [trade_summary_eod['Total Trades']], color='skyblue')
        plt.xlabel('Count')
        plt.ylabel('Total Trades')
        plt.title(f'Total Trades for Best PNL Combination (EOD)\n{folder_name}')
        plt.tight_layout()
        plt.show()
        plt.savefig(f'{output_dir_eod}/{folder_name}_total_trades_bar_chart.png')
        plt.close()
    
        plt.figure(figsize=(12, 8))
        plt.barh(['Max Drawdown'], [trade_summary_eod['Max Drawdown']], color='red')
        plt.xlabel('Max Drawdown')
        plt.ylabel('Max Drawdown Value')
        plt.title(f'Max Drawdown for Best PNL Combination (EOD)\n{folder_name}')
        plt.tight_layout()
        plt.show()
        plt.savefig(f'{output_dir_eod}/{folder_name}_max_drawdown_bar_chart.png')
        plt.close()
    
        plt.figure(figsize=(12, 8))
        plt.barh(top_40_combinations_eod['Profit Target'].astype(str) + " / " + top_40_combinations_eod['Stop Loss'].astype(str),
                 top_40_combinations_eod['Total Profit'], color='skyblue')
        plt.xlabel('Total Profit')
        plt.ylabel('Profit Target / Stop Loss')
        plt.title(f'Top 40 Profit Target and Stop Loss Combinations (EOD)\n{folder_name}')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()
        plt.savefig(f'{output_dir_eod}/{folder_name}_top_40_combinations.png')
        plt.close()

    # Combine and save results for carry over trades
    if pnl_results_carry_over:
        combined_pnl_results_carry_over = pd.concat(pnl_results_carry_over, ignore_index=True)
        combined_pnl_results_carry_over = combined_pnl_results_carry_over.groupby(['Profit Target', 'Stop Loss'], as_index=False)['Total Profit'].sum()
        combined_trade_results_carry_over = pd.concat(trade_results_combined_carry_over, ignore_index=True)
        combined_all_results_carry_over = pd.concat(all_combined_results_carry_over, ignore_index=True)

        output_dir_carry_over = f'Outputs/DayTrade/SOSI/5min/{folder_name}_carry_over'
        os.makedirs(output_dir_carry_over, exist_ok=True)
        
        combined_pnl_results_carry_over.to_csv(f'{output_dir_carry_over}/{folder_name}_pnl_analysis.csv', index=False)
        combined_trade_results_carry_over.to_csv(f'{output_dir_carry_over}/{folder_name}_trade_results.csv', index=False)
        combined_all_results_carry_over.to_csv(f'{output_dir_carry_over}/{folder_name}_all_combined_results.csv', index=False)
        
        best_combination_carry_over = combined_pnl_results_carry_over.loc[combined_pnl_results_carry_over['Total Profit'].idxmax()]
        best_combination_df_carry_over = pd.DataFrame([best_combination_carry_over])
        best_combination_df_carry_over.to_csv(f'{output_dir_carry_over}/{folder_name}_best_combination.csv', index=False)
        
        top_40_combinations_carry_over = combined_pnl_results_carry_over.sort_values(by='Total Profit', ascending=False).head(40)
        top_40_combinations_carry_over.to_csv(f'{output_dir_carry_over}/{folder_name}_top_40_combinations.csv', index=False)
        
        best_trade_results_carry_over = combined_trade_results_carry_over[(combined_trade_results_carry_over['Target Profit'] == best_combination_carry_over['Profit Target']) & 
                                                                          (combined_trade_results_carry_over['Stop Loss'] == best_combination_carry_over['Stop Loss'])]
        trade_summary_carry_over = summarize_trade_results(best_trade_results_carry_over)
        trade_summary_df_carry_over = pd.DataFrame([trade_summary_carry_over])
        trade_summary_df_carry_over.to_csv(f'{output_dir_carry_over}/{folder_name}_trade_summary.csv', index=False)
        
        labels = ['Target Hit', 'Stop Loss Hit', 'End of Day Exit']
        sizes = [trade_summary_carry_over['Target Hit'], trade_summary_carry_over['Stop Loss Hit'], trade_summary_carry_over['End of Day Exit']]
        colors = ['lightgreen', 'lightcoral', 'lightskyblue']
        plt.figure(figsize=(8, 8))
        plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
        plt.title(f'Trade Summary for Best PNL Combination (Carry Over)\n{folder_name}')
        plt.axis('equal')
        plt.show()
        plt.savefig(f'{output_dir_carry_over}/{folder_name}_trade_summary_pie_chart.png')
        plt.close()
        
        plt.figure(figsize=(12, 8))
        plt.barh(['Total Trades'], [trade_summary_carry_over['Total Trades']], color='skyblue')
        plt.xlabel('Count')
        plt.ylabel('Total Trades')
        plt.title(f'Total Trades for Best PTotal Trades for Best PNL Combination (Carry Over)\n{folder_name}')
        plt.tight_layout()
        plt.show()
        plt.savefig(f'{output_dir_carry_over}/{folder_name}_total_trades_bar_chart.png')
        plt.close()
    
        plt.figure(figsize=(12, 8))
        plt.barh(['Max Drawdown'], [trade_summary_carry_over['Max Drawdown']], color='red')
        plt.xlabel('Max Drawdown')
        plt.ylabel('Max Drawdown Value')
        plt.title(f'Max Drawdown for Best PNL Combination (Carry Over)\n{folder_name}')
        plt.tight_layout()
        plt.show()
        plt.savefig(f'{output_dir_carry_over}/{folder_name}_max_drawdown_bar_chart.png')
        plt.close()
    
        plt.figure(figsize=(12, 8))
        plt.barh(top_40_combinations_carry_over['Profit Target'].astype(str) + " / " + top_40_combinations_carry_over['Stop Loss'].astype(str),
                 top_40_combinations_carry_over['Total Profit'], color='skyblue')
        plt.xlabel('Total Profit')
        plt.ylabel('Profit Target / Stop Loss')
        plt.title(f'Top 40 Profit Target and Stop Loss Combinations (Carry Over)\n{folder_name}')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()
        plt.savefig(f'{output_dir_carry_over}/{folder_name}_top_40_combinations.png')
        plt.close()

print("Analysis complete.")




Processing NSE_NIFTY240627C23500, 5.csv in NIFTY...
No buy signals found before 9:30 AM IST for NSE_NIFTY240627C23500, 5.csv. Skipping...
Processing NSE_NIFTY240627C23600, 5.csv in NIFTY...
No buy signals found before 9:30 AM IST for NSE_NIFTY240627C23600, 5.csv. Skipping...
Processing NSE_NIFTY240627C23700, 5.csv in NIFTY...
No buy signals found before 9:30 AM IST for NSE_NIFTY240627C23700, 5.csv. Skipping...
Processing NSE_NIFTY240627C23750, 5.csv in NIFTY...
No buy signals found before 9:30 AM IST for NSE_NIFTY240627C23750, 5.csv. Skipping...
Processing NSE_NIFTY240627C23800, 5.csv in NIFTY...
No buy signals found before 9:30 AM IST for NSE_NIFTY240627C23800, 5.csv. Skipping...
Processing NSE_NIFTY240627P23600, 5.csv in NIFTY...
No buy signals found before 9:30 AM IST for NSE_NIFTY240627P23600, 5.csv. Skipping...
Processing NSE_NIFTY240627P23700, 5.csv in NIFTY...
No buy signals found before 9:30 AM IST for NSE_NIFTY240627P23700, 5.csv. Skipping...
Processing NSE_NIFTY240627P23800, 

C:\temp\ipykernel_16264\1091108595.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_signals['time_ist'] = pd.to_datetime(buy_signals['time'], unit='s') + timedelta(hours=5, minutes=30)
C:\temp\ipykernel_16264\1091108595.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_signals['time_ist'] = pd.to_datetime(buy_signals['time'], unit='s') + timedelta(hours=5, minutes=30)


No buy signals found before 9:30 AM IST for NSE_NIFTY240704C23800, 5.csv. Skipping...
Processing NSE_NIFTY240704C23900, 5.csv in NIFTY...
No buy signals found before 9:30 AM IST for NSE_NIFTY240704C23900, 5.csv. Skipping...
Processing NSE_NIFTY240704C24000, 5.csv in NIFTY...


C:\temp\ipykernel_16264\1091108595.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_signals['time_ist'] = pd.to_datetime(buy_signals['time'], unit='s') + timedelta(hours=5, minutes=30)
C:\temp\ipykernel_16264\1091108595.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_signals['time_ist'] = pd.to_datetime(buy_signals['time'], unit='s') + timedelta(hours=5, minutes=30)


No buy signals found before 9:30 AM IST for NSE_NIFTY240704C24000, 5.csv. Skipping...
Processing NSE_NIFTY240704C24050, 5.csv in NIFTY...
No buy signals found before 9:30 AM IST for NSE_NIFTY240704C24050, 5.csv. Skipping...
Processing NSE_NIFTY240704C24100, 5.csv in NIFTY...


C:\temp\ipykernel_16264\1091108595.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_signals['time_ist'] = pd.to_datetime(buy_signals['time'], unit='s') + timedelta(hours=5, minutes=30)
C:\temp\ipykernel_16264\1091108595.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_signals['time_ist'] = pd.to_datetime(buy_signals['time'], unit='s') + timedelta(hours=5, minutes=30)


No buy signals found before 9:30 AM IST for NSE_NIFTY240704C24100, 5.csv. Skipping...
Processing NSE_NIFTY240704C24150, 5.csv in NIFTY...
